# Purpose of this notebook
- Use Bidirectional combine with SimpleRNN, GRU, LSTM to solve mnist

# Import

In [1]:
from tensorflow import keras # type: ignore
from tensorflow.keras import layers, regularizers # type: ignore
from tensorflow.keras.datasets import mnist # type: ignore

# Load data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


# Before normalize, range in [0, 255]

In [3]:
print(x_train[0][10][10: 15])
print(x_test[0][10][10: 15])

[  1 154 253  90   0]
[ 0 17 66 14 67]


# Normalize
- Note: Run following cell only once

In [4]:
x_train = x_train / 255
x_test = x_test / 255

# After normalize, range in [0, 1]

In [5]:
print(x_train[0][10][10: 15])
print(x_test[0][10][10: 15])

[0.00392157 0.60392157 0.99215686 0.35294118 0.        ]
[0.         0.06666667 0.25882353 0.05490196 0.2627451 ]


# Keras Sequential API using Simple RNN

In [6]:
model = keras.Sequential([
    layers.Input(shape=(None, 28), name='input_layer'),
    layers.Bidirectional(
        layers.SimpleRNN(
            256,
            return_sequences=True,
            activation='tanh',
            name='rnn_layer1',
        )
    ),
    layers.Bidirectional(
        layers.SimpleRNN(
            256,
            activation='tanh',
            name='rnn_layer2',
        )
    ),
    layers.Dense(10, name='output_layer'),
], name="sequential_model")

# model.summary 

In [7]:
print(model.summary())

Model: "sequential_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, None, 512)      │       145,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 512)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 544,778 (2.08 MB)

 Trainable params: 544,778 (2.08 MB)

 Non-trainable params: 0 (0.00 B)

None


# model.compile

In [8]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

# Note
- If output layer `layers.Dense(10)` then use `from_logits=True`
- If output layer `layers.Dense(10, activation='softmax')` then use `from_logits=False`, or just simply remove `from_logits=False`

# Train the model using model.fit

In [9]:
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
1875/1875 - 61s - 32ms/step - accuracy: 0.8988 - loss: 0.3376
Epoch 2/5
1875/1875 - 55s - 29ms/step - accuracy: 0.9327 - loss: 0.2254
Epoch 3/5
1875/1875 - 56s - 30ms/step - accuracy: 0.9334 - loss: 0.2218
Epoch 4/5
1875/1875 - 56s - 30ms/step - accuracy: 0.9398 - loss: 0.2008
Epoch 5/5
1875/1875 - 56s - 30ms/step - accuracy: 0.9472 - loss: 0.1777


# Note
- model.fit above run in 4m43s (longest time so far)
- And achieve accuracy 0.9472 (good)

# Bidirectional can combine with SimpleRNN, GRU, LSTM
- layers.Bidirectional(layers.SimpleRNN(64))
- layers.Bidirectional(layers.GRU(64))
- layers.Bidirectional(layers.LSTM(64))

# Evaluate test data using model.evaluate

In [10]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 4s - 14ms/step - accuracy: 0.9361 - loss: 0.2103


[0.2103237360715866, 0.9361000061035156]